# Projet Deep Learning: reconnaissance de cartes

## Chargement des données

### Création du DataFrame et des 3 ensembles train, test et validation

In [ ]:
import os
import pandas as pd
from google.colab import drive
import numpy as np
import matplotlib.pyplot as plt

colab = True

if colab:
  drive.mount("/content/drive")
  folder = "/content/drive/MyDrive/dataset-cartes/classes"
else:
  folder = "dataset/classes"

card_type = ["clubs", "diamonds", "hearts", "spades"]
card_type_to_idx = {card_type[i]: i for i in range(len(card_type))}
card_number = ["seven", "eight", "nine", "ten", "jack", "queen", "king", "ace"]
card_number_to_idx = {card_number[i]: i for i in range(len(card_number))}
labels = [f"{n}_{t}" for n in card_number for t in card_type]
labels_to_idx = {labels[i]: i for i in range(len(labels))}
# labels_to_idx = {labels[i].split("_")[0]: i for i in range(len(labels))}

image_size = 128
num_classes = 32
num_value = 8
num_color = 4

In [ ]:
def get_dataframe_from_folder(folder, classes):
    """
    Create a dataframe containing information about the dataset from a folder.
    Parameters
    ----------
    folder: string
      path to the root folder
    classes: []string
      names of the classes
    """
    df = pd.DataFrame(columns=["path", "number", "type", "label"])
    for label in classes:
        path_folder_label = os.path.join(folder, label)
        for img in os.listdir(path_folder_label):
            path = os.path.join(folder, label, img)
            s = label.split("_")
            card_number = s[0]
            card_type = s[1]
            df.loc[len(df)] = [path, card_number, card_type, label]
    return df

In [ ]:
def extract_number_each_class(df, classes, n=70):
  """
  Extract a number of data for each classes.
  Parameters
  ----------
  df: DataFrame
    data
  classes: []string
    names of the classes
  n: int, optional
    number of data extract from each class
  """
  new_df = pd.DataFrame(columns=["path", "number", "type", "label"])
  m = {c: 0 for c in classes}
  # for label in classes:
  #   for _, row in df.iterrows():
  #     if m[row["label"]] < 40:
  #       new_df.loc[len(new_df)] = row
  for label in classes:
    for _, row in df.iterrows():
        if m[label] < n and row["label"] == label:
            new_df.loc[len(new_df)] = row
            m[label] += 1
  return new_df

In [ ]:
def print_stat(df):
    """
    Print the total number of cards and the number of cards for each label.
    Parameters
    ----------
    df: DataFrame
      data
    """
    print(f"Il y a {len(df)} cartes.")
    sorted_df = df.groupby('label').count().sort_values('path', ascending=False)["path"]
    print(sorted_df)

In [ ]:
import math
import random

def split_train_test_validation(df, ratio=[0.70, 0.15, 0.15], seed=10):
    """
    Split the dataframe in 3 dataframe (train, test and validation)
    Parameters
    ----------
    df: DataFrame
      data
    ratio: []int, optional
      ratio of data for train, test and validation dataset
    seed: int, optional
      seed for the random state
    """
    n = len(df)
    df = df.sample(n=n, random_state=seed)
    last_train = math.floor(n * ratio[0])
    last_test = last_train + 1 + math.floor(n * ratio[1])
    df_train = df[0:last_train]
    df_test = df[last_train:last_test]
    df_val = df[last_test:]
    n_train = len(df_train)
    n_test = len(df_test)
    n_val = len(df_val)
    print(f"train: {n_train}, test: {n_test}, val: {n_val}, tot: {n_train + n_test + n_val}")
    return df_train, df_test, df_val

### Chargement des images

In [ ]:
import PIL
from PIL import Image
from tqdm import tqdm

def load_data(df, labels_to_idx, image_size=128):
    """
    Split the dataframe in 3 dataframe (train, test and validation)
    Parameters
    ----------
    df: DataFrame
      data
    labels_to_idx: map[string]int
      map a class name to a index
    image_size: int, optional
      image size
    """
    n = len(df)
    x = np.zeros((n, image_size, image_size, 3))
    y = np.zeros((n, 1))

    index = 0
    for _, row in tqdm(df.iterrows(), total=n):
        img = Image.open(row["path"])
        img = img.convert('RGB')
        img = img.resize((image_size, image_size))
        x[index] = np.asarray(img)
        y[index] = labels_to_idx[row["label"]]
        # y[index] = labels_to_idx[row["label"].split("_")[0]]
        index += 1
    return x, y

### Génération ou chargement des données

In [ ]:
import h5py

generate_dataset = False

if generate_dataset:

  df_raw = get_dataframe_from_folder(folder, labels)
  df_raw.head()
  n = 70
  df = extract_number_each_class(df_raw,labels,n)
  print("# df_raw:")
  print_stat(df_raw)
  print("\n# df:")
  print_stat(df)

  df_train, df_test, df_val = split_train_test_validation(df)#, seed=random.randrange(1000))
  x_train, y_train = load_data(df_train, labels_to_idx)
  print(x_train.shape, y_train.shape)

  x_test, y_test = load_data(df_test, labels_to_idx)
  print(x_test.shape, y_test.shape)

  x_val, y_val = load_data(df_val, labels_to_idx)
  print(x_val.shape, y_val.shape)

  hf = h5py.File(folder + "/dataset.h5", 'a')
  hf.create_dataset('x_train', data=x_train)
  hf.create_dataset('y_train', data=y_train)
  hf.create_dataset('x_test', data=x_test)
  hf.create_dataset('y_test', data=y_test)
  hf.create_dataset('x_val', data=x_val)
  hf.create_dataset('y_val', data=y_val)
  hf.close()
  df_train.to_hdf(folder + "/dataset.h5", key='df_train', mode='a')
  df_test.to_hdf(folder + "/dataset.h5", key='df_test', mode='a')
  df_val.to_hdf(folder + "/dataset.h5", key='df_val', mode='a')
else:
  hf = h5py.File(folder + "/dataset.h5", 'r')
  x_train = hf.get('x_train')[:]
  y_train = hf.get('y_train')[:]
  x_test = hf.get('x_test')[:]
  y_test = hf.get('y_test')[:]
  x_val = hf.get('x_val')[:]
  y_val = hf.get('y_val')[:]
  hf.close()
  df_train = pd.read_hdf(folder + "/dataset.h5", 'df_train')
  df_test = pd.read_hdf(folder + "/dataset.h5", 'df_test')
  df_val = pd.read_hdf(folder + "/dataset.h5", 'df_val')

### Augmentation des données

In [ ]:
import albumentations as A
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import matplotlib.pyplot as plt

# Déclarer un pipeline d'augmentation
transforms = A.Compose([
    # Transformation en noir et blanc
    A.ToGray(p = 0.6),
    # Ajouter du CoarseDropout

    #max_holes : Le nombre maximum de trous à créer dans l'image. Un trou est une zone masquée.
    #max_height : La hauteur maximale d'un trou.
    #max_width : La largeur maximale d'un trou.
    #min_holes : Le nombre minimum de trous à créer dans l'image.
    #min_height : La hauteur minimale d'un trou.
    #min_width : La largeur minimale d'un trou.
    #fill_value : La valeur utilisée pour remplir les trous.
    #p : La probabilité d'appliquer cette transformation à une image donnée.

    A.CoarseDropout(max_holes=10, max_height=30, max_width=30, min_holes=2, min_height=5, min_width=5, fill_value=0, p=0.8),

    # Autres transformations selon vos besoins
    A.ChannelShuffle(p=0.5)
    #A.HorizontalFlip(p=0.5),
])

def apply_custom_transform(image):
    return transforms(image=image)['image']

train_datagen = ImageDataGenerator(
    preprocessing_function=apply_custom_transform,

    # Ajoutez d'autres paramètres d'augmentation si nécessaire
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    rescale=1./255
)

# Utilisez le même générateur pour la validation si vous le souhaitez
val_datagen = ImageDataGenerator(
    rescale=1./255
)

# Exemple d'utilisation du générateur de données
example_x, example_y = train_datagen.flow(x_train, y_train, batch_size=1).next()
plt.imshow(example_x[0])  # Afficher l'image en noir et blanc
plt.title(labels[int(example_y[0])])
plt.show()


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator


import numpy as np
from PIL import ImageEnhance, Image


# Initialiser les générateurs de données avec la fonction de transformation personnalisée
train_datagen = ImageDataGenerator(

    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=False,
    rescale=1./255
)

val_datagen = ImageDataGenerator(
    rescale=1./255
)

# Exemple d'utilisation du générateur de données
example_x, example_y = train_datagen.flow(x_train, y_train, batch_size=1).next()
plt.imshow(example_x[0])
plt.title(labels[int(example_y[0])])
plt.show()


In [ ]:
#train_datagen.fit(x_train)

#x_train_gen, y_train_gen = train_datagen.flow(x_train, y_train, batch_size=1)
#print(x_train_gen)

### Visualisation des données

In [ ]:
import matplotlib.pyplot as plt

def show_imgs(x, y, labels):
    """
    Display 9 images.
    Parameters
    ----------
    x: numpy.array()
      images
    y: numpy.array()
      labels of images
    labels: []string
      names of labels
    """
    plt.figure(figsize=(12,12))
    shuffle_indices = np.random.permutation(x.shape[0])
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        img = x[shuffle_indices[i]]
        plt.title(labels[int(y[shuffle_indices[i]])])
        plt.imshow(img/255)
    plt.tight_layout()
    plt.show()

show_imgs(x_train, y_train, labels)

In [ ]:
from sklearn.metrics import confusion_matrix as sk_confusion_matrix
import itertools

def show_imgs_prediction(model, x, y, labels):
    """
    Display prediction of 9 random images with a model.
    Parameters
    ----------
    model: keras model
    x: numpy.array()
      images
    y: numpy.array()
      labels of images
    labels: []string
      names of labels
    """
    plt.figure(figsize=(12,12))
    shuffle_indices = np.random.permutation(x.shape[0])
    for i in range(9):
        plt.subplot(3, 3, i + 1)
        img = x[shuffle_indices[i]]
        pred = model.predict(np.array([img,]))
        label_pred = np.argmax(pred)
        print(label_pred)
        plt.title(f"predict: {labels[label_pred]}, label:{labels[int(y[shuffle_indices[i]])]}")
        plt.imshow(img/255)
    plt.tight_layout()
    plt.show()

def plot_history(history):
  """
  Plot accuracy and loss.
  Parameters
  ----------
  history: keras history
    the history
  """
  acc = history.history['accuracy']
  val_acc = history.history['val_accuracy']
  loss = history.history['loss']
  val_loss = history.history['val_loss']

  epochs = range(len(acc))

  plt.plot(epochs, acc, 'b', linestyle="--",label='Training acc')
  plt.plot(epochs, val_acc, 'g', label='Validation acc')
  plt.title('Training and validation accuracy')
  plt.legend()

  plt.figure()

  plt.plot(epochs, loss, 'b', linestyle="--",label='Training loss')
  plt.plot(epochs, val_loss,'g', label='Validation loss')
  plt.title('Training and validation loss')
  plt.legend()

  plt.show()

def confusion_matrix(y, y_pred, labels, figsize=(10,10), text_size=8):
  print(len(np.unique(y)))
  print(len(np.unique(y_pred)))
  cm = sk_confusion_matrix(y, y_pred)
  cm_normalized = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
  n = len(labels)

  fig, ax = plt.subplots(figsize=figsize)
  color = ax.matshow(cm, cmap=plt.cm.Blues)
  fig.colorbar(color)

  arange = np.arange(n)
  labels = [f"{l} ({i})" for i, l in enumerate(labels)]
  ax.set(title="Confusion matrix", xlabel="Pred", ylabel="True",
        xticks=arange, yticks=arange, xticklabels=arange, yticklabels=labels)

  ax.xaxis.set_label_position("bottom")
  ax.xaxis.tick_bottom()

  threshold = (cm.max() + cm.min()) / 2

  for i, j in itertools.product(range(n), range(n)):
      if i < cm.shape[0] and j < cm.shape[1]:
        v = cm[i,j]
      else:
        v = 0
      plt.text(j, i, f"{v}",
                horizontalalignment="center",
                verticalalignment="center",
                color="white" if v > threshold else "black",
                size=text_size)

### Fonction pour split la couleur et la valeur des labels

In [ ]:
def split_y(y):
  """
  Split 32 classes labels into 8 and 4 classes labels.
  Parameters
  ----------
  y: numpy.array()
    labels of images
  """
  y_split = np.zeros((len(y),2))
  for i in range(len(y)):
    y_split[i,0] = (y[i] // 4)[0]
    y_split[i,1] = (y[i] % 4)[0]
  return y_split

def get_accuracy_split(y, y_pred_value, y_pred_color):
  """
  Compute accuracy on the predicted labels when outputs is splited.
  Parameters
  ----------
  y: numpy.array()
    predicted albels
  y_pred_value: numpy.array()
    value labels of images
  y_pred_color: numpy.array()
    color labels of images
  """
  y = y.astype(int)
  acc_color = 0
  acc_value = 0
  acc = 0
  for i in range(len(y)):
    y_value = y[i,0]
    y_color = y[i,1]
    pred_value = np.argmax(y_pred_value[i])
    pred_color = np.argmax(y_pred_color[i])
    if y_value == pred_value:
      acc_value += 1
    if y_color == pred_color:
      acc_color += 1
    if y_value == pred_value and y_color == pred_color:
      acc += 1
  print(f"acc_value = {acc_value/len(y):2f}, acc_color = {acc_color/len(y):2f}, acc = {acc/len(y):2f}")

## Modèle et entrainement

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Input
from keras import regularizers
from tensorflow.keras import optimizers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

#### Callback

In [ ]:
## Callbacks
checkpoint = ModelCheckpoint('best_model.h5',verbose=1,save_best_only=True,save_weights_only=True)
earlystopping = EarlyStopping(patience=10,verbose=1)

#### One Hot encoding

In [ ]:
y_train_onehot = to_categorical(y_train, num_classes)
y_test_onehot = to_categorical(y_test,num_classes)
y_val_onehot = to_categorical(y_val, num_classes)

In [ ]:
y_train_split = split_y(y_train)
y_test_split = split_y(y_test)
y_val_split = split_y(y_val)
y_train_split_value = y_train_split[:,0]
y_train_split_color = y_train_split[:,1]
y_test_split_value = y_test_split[:,0]
y_test_split_color = y_test_split[:,1]
y_val_split_value = y_val_split[:,0]
y_val_split_color = y_val_split[:,1]
y_train_split_onehot_value = to_categorical(y_train_split[:,0], num_value)
y_train_split_onehot_color = to_categorical(y_train_split[:,1], num_color)
y_test_split_onehot_value = to_categorical(y_test_split[:,0], num_value)
y_test_split_onehot_color = to_categorical(y_test_split[:,1], num_color)
y_val_split_onehot_value = to_categorical(y_val_split[:,0], num_value)
y_val_split_onehot_color = to_categorical(y_val_split[:,1], num_color)

### Modèle simple

#### Entrainement

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=(image_size,image_size,3)))
model.add(Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=(image_size,image_size,3)))
model.add(MaxPool2D(pool_size=2))
model.add(Conv2D(filters=64, kernel_size=3, activation="relu", input_shape=(image_size,image_size,3)))
model.add(Conv2D(filters=64, kernel_size=3, activation="relu", input_shape=(image_size,image_size,3)))
model.add(MaxPool2D(pool_size=2))
model.add(Conv2D(filters=128, kernel_size=3, activation="relu", input_shape=(image_size,image_size,3)))
model.add(Conv2D(filters=128, kernel_size=3, activation="relu", input_shape=(image_size,image_size,3)))
model.add(MaxPool2D(pool_size=3))
model.add(Flatten())
#model.add(Dense(1024, activation="relu"))
model.add(Dense(512, activation="relu"))
model.add(Dense(256, activation="relu"))
model.add(Dense(32, activation="softmax"))
#model.add(Dense(512, activation="relu",kernel_regularizer=regularizers.L1L2(l1=0.01, l2=0.02)))
#model.add(Dense(256, activation="relu",kernel_regularizer=regularizers.L1L2(l1=0.01, l2=0.02)))
#model.add(Dense(32, activation="softmax",kernel_regularizer=regularizers.L1L2(l1=0.01, l2=0.02)))

In [ ]:
model.compile(loss="categorical_crossentropy",
              optimizer=optimizers.Adam(learning_rate=3e-4),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
# Sans augmentation:
# history = model.fit(
#     x_train/255,
#     y_train_onehot,
#     validation_data=(x_val/255, y_val_onehot),
#     epochs=30,
#     batch_size=32
# )

# Avec augmentation:
history = model.fit(train_datagen.flow(x_train, y_train_onehot, batch_size=32),
                    validation_data=val_datagen.flow((x_val, y_val_onehot)),
                    epochs=70, callbacks=[checkpoint])

In [ ]:
!pip install visualkeras
# from keras.utils import plot_model
import visualkeras

# plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True, rankdir="LR")

visualkeras.layered_view(model).show() # display using your system viewer
# visualkeras.layered_view(model, to_file='output.png') # write to disk
# visualkeras.layered_view(model, to_file='output.png').show() # write and show

visualkeras.layered_view(model, legend=True)




#### Visualisation de nos résultats

##### Prediction et evaluation de l'ensemble de test

In [ ]:
model.evaluate(x_test/255, y_test_onehot)
y_pred = np.argmax(model.predict(x_test/255), axis=1)

In [ ]:
# accuracy par classe
liste_bonnes_classifs = [0 for i in range(32)]
liste_nb_classifs = [0 for i in range(32)]
for i in range(len(y_pred)):
  if y_pred[i] == y_test[i,0]:
    liste_bonnes_classifs[int(y_test[i,0])] = liste_bonnes_classifs[int(y_test[i,0])] + 1
  liste_nb_classifs[int(y_test[i,0])] = liste_nb_classifs[int(y_test[i,0])] + 1
liste_accuracy = [liste_bonnes_classifs[i]/liste_nb_classifs[i] for i in range(32)]
for i in range(32):
  print(labels[i].split("_")[0] + "\_" + labels[i].split("_")[1] + " : " + str(round(liste_accuracy[i]*100,2)) + " \% " + "- " + str(liste_nb_classifs[i]) + " images de test \\\\")

In [ ]:
plot_history(history)

In [ ]:
show_imgs_prediction(model, x_test, y_test, labels)

##### Matrice de confusion

In [ ]:
confusion_matrix(y_test, y_pred, labels)

### Modèle simple avec 2 sorties (couleur et valeur)

In [ ]:
input_layer = Input(shape=(image_size, image_size, 3))
conv2d = Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=(image_size,image_size,3))(input_layer)
conv2d = Conv2D(filters=32, kernel_size=3, activation="relu", input_shape=(image_size,image_size,3))(conv2d)
maxpool2d = MaxPool2D(pool_size=2)(conv2d)
conv2d = Conv2D(filters=64, kernel_size=3, activation="relu", input_shape=(image_size,image_size,3))(maxpool2d)
conv2d = Conv2D(filters=64, kernel_size=3, activation="relu", input_shape=(image_size,image_size,3))(conv2d)
maxpool2d = MaxPool2D(pool_size=2)(conv2d)
conv2d = Conv2D(filters=128, kernel_size=3, activation="relu", input_shape=(image_size,image_size,3))(maxpool2d)
conv2d = Conv2D(filters=128, kernel_size=3, activation="relu", input_shape=(image_size,image_size,3))(conv2d)
maxpool2d = MaxPool2D(pool_size=2)(conv2d)
flatten = Flatten()(maxpool2d)
#modelv2.add(Dense(1024, activation="relu"))
dense = Dense(512, activation="relu")(flatten)
dense = Dense(256, activation="relu")(dense)

output_value = Dense(8, activation="softmax", name="value")(dense)
output_color = Dense(4, activation="softmax", name="color")(dense)

output = [output_value, output_color]
modelv2 = Model(input_layer, output)

In [ ]:
modelv2.summary()

#### Entrainement

In [ ]:
loss = ["categorical_crossentropy", "categorical_crossentropy"]
metrics = [["accuracy"], ["accuracy"]]
loss_weights = [2/3, 1/3]

In [ ]:
modelv2.compile(optimizer=optimizers.Adam(lr=1e-4),
              loss=loss,
              metrics=metrics,
              loss_weights=loss_weights)



historyv2 = modelv2.fit(
    x_train,
    [y_train_split_onehot_value, y_train_split_onehot_color],
    epochs=50,
    batch_size=128,
    validation_data=(x_val, [y_val_split_onehot_value, y_val_split_onehot_color])
)


#### Visualisation de nos résultats

In [ ]:
def plot_historyv2(history):
    """
    Plot accuracy and loss for a model with two outputs.
    Parameters
    ----------
    history: keras history
        the history object returned by model.fit
    """
    # Accuracy and loss for the first output (value)
    acc_value = history.history['value_accuracy']
    val_acc_value = history.history['val_value_accuracy']
    loss_value = history.history['value_loss']
    val_loss_value = history.history['val_value_loss']

    # Accuracy and loss for the second output (color)
    acc_color = history.history['color_accuracy']
    val_acc_color = history.history['val_color_accuracy']
    loss_color = history.history['color_loss']
    val_loss_color = history.history['val_color_loss']

    epochs = range(len(acc_value))

    plt.figure(figsize=(12, 6))

    # Plot accuracy for value
    plt.subplot(2, 2, 1)
    plt.plot(epochs, acc_value, 'b', linestyle="--", label='Training value acc')
    plt.plot(epochs, val_acc_value, 'g', label='Validation value acc')
    plt.title('Training and validation value accuracy')
    plt.legend()

    # Plot accuracy for color
    plt.subplot(2, 2, 2)
    plt.plot(epochs, acc_color, 'b', linestyle="--", label='Training color acc')
    plt.plot(epochs, val_acc_color, 'g', label='Validation color acc')
    plt.title('Training and validation color accuracy')
    plt.legend()

    # Plot loss for value
    plt.subplot(2, 2, 3)
    plt.plot(epochs, loss_value, 'b', linestyle="--", label='Training value loss')
    plt.plot(epochs, val_loss_value, 'g', label='Validation value loss')
    plt.title('Training and validation value loss')
    plt.legend()

    # Plot loss for color
    plt.subplot(2, 2, 4)
    plt.plot(epochs, loss_color, 'b', linestyle="--", label='Training color loss')
    plt.plot(epochs, val_loss_color, 'g', label='Validation color loss')
    plt.title('Training and validation color loss')
    plt.legend()

    plt.tight_layout()
    plt.show()


In [ ]:
y_pred_value, y_pred_color = modelv2.predict(x_test)

In [ ]:
get_accuracy_split(y_test_split, y_pred_value, y_pred_color)

In [ ]:
plot_historyv2(historyv2)

##### Prediction et evaluation de l'ensemble de test

##### Matrice de confusion

In [ ]:
y_test_value = y_test_split[:,0]
y_test_color = y_test_split[:,1]
confusion_matrix(y_test_value, np.argmax(y_pred_value, axis=1), card_number)

In [ ]:
confusion_matrix(y_test_color, np.argmax(y_pred_color, axis=1), card_type)

### Transfer learning


In [ ]:
from tensorflow.keras.applications import ResNet50V2, VGG16, MobileNetV2
from tensorflow.keras.layers import Flatten, Dense, Dropout
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

# Définir le callback ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1)

# Assurez-vous d'ajouter le checkpoint callback aussi si ce n'est pas déjà fait
checkpoint = ModelCheckpoint(filepath='best_model.h5', monitor='val_loss', save_best_only=True, verbose=1)


conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(image_size, image_size, 3))

conv_base.trainable = False

#### Entrainement (fine tuning)

In [ ]:
model_translearn = Sequential()
model_translearn.add(conv_base)
model_translearn.add(Flatten())
model_translearn.add(Dense(1024, activation="relu"))
model_translearn.add(Dense(32, activation="softmax"))
model_translearn.summary()

initial_epochs = 20


# Compiler le modèle avec un taux d'apprentissage plus élevé
model_translearn.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
                         loss='categorical_crossentropy',
                         metrics=['accuracy'])

# COMPLETER AVEC LES TENSEURS SUR LESQUELS EFFECTUER L'APPRENTISSAGE


history = model_translearn.fit(train_datagen.flow(x_train, y_train_onehot, batch_size=32),
                    validation_data=val_datagen.flow((x_val, y_val_onehot)),
                    epochs=initial_epochs, callbacks=[checkpoint, reduce_lr])


conv_base.trainable = True



train_generator = train_datagen.flow(x_train, y_train_onehot, batch_size=32)

validation_generator =  val_datagen.flow((x_val, y_val_onehot))

# Fonction pour dégeler progressivement les couches et réentraîner le modèle
def fine_tune_model(model, conv_base, train_generator, validation_generator, initial_epochs, additional_epochs, learning_rate, layers_to_unfreeze):
    # Dégeler les couches
    conv_base.trainable = True
    for layer in conv_base.layers[:-layers_to_unfreeze]:
        layer.trainable = False

    # Recompiler le modèle avec un taux d'apprentissage plus bas
    model.compile(optimizer=optimizers.Adam(learning_rate=learning_rate),
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    # Continuer l'entraînement
    history_fine = model.fit(train_generator,
                    validation_data=validation_generator,
                    epochs=initial_epochs + additional_epochs,
                    steps_per_epoch=len(x_train) // 32,
                    validation_steps=len(x_val) // 32,
                    callbacks=[checkpoint, reduce_lr])

    return history_fine

# Définir les paramètres pour le fine-tuning progressif

total_epochs = 50
fine_tuning_steps = [
    {"epochs": 10, "learning_rate": 1e-5, "layers_to_unfreeze": 50},
    {"epochs": 10, "learning_rate": 1e-6, "layers_to_unfreeze": 100},
    {"epochs": 10, "learning_rate": 1e-7, "layers_to_unfreeze": len(conv_base.layers)}
]

# Exécuter le fine-tuning progressif
current_epochs = initial_epochs
for step in fine_tuning_steps:
    history_fine = fine_tune_model(model_translearn, conv_base, train_generator, validation_generator,
                                   current_epochs, step["epochs"], step["learning_rate"], step["layers_to_unfreeze"])
    current_epochs += step["epochs"]


In [ ]:
plot_history(history)

In [ ]:
confusion_matrix(y_test, y_pred, labels)

### Transfer learning avec 2 outputs

#### Entrainement

In [ ]:
conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(image_size, image_size, 3))

conv_base.trainable = False

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

input_layer = Input(shape=(image_size, image_size, 3))
x = conv_base(input_layer, training=False)
x = Flatten()(x)
prev_layer = Dense(512, activation="relu")(x)

output_value = Dense(8, activation="softmax", name="value")(prev_layer)
output_color = Dense(4, activation="softmax", name="color")(prev_layer)

output = [output_value, output_color]
model_translearn_split = Model(input_layer, output)

model_translearn_split.summary()

In [ ]:
model_translearn_split.compile(optimizer=optimizers.Adam(lr=1e-4),
              loss=loss,
              metrics=metrics,
              loss_weights=loss_weights)

#  COMPLETER AVEC LES TENSEURS SUR LESQUELS EFFECTUER L'APPRENTISSAGE
# history = model_translearn_split.fit(train_datagen.flow(x_train, [y_train_split_onehot_value, y_train_split_onehot_color], batch_size=128),
#                     epochs=50, callbacks=[checkpoint])

history = model_translearn_split.fit(
    x_train,
    [y_train_split_onehot_value, y_train_split_onehot_color],
    epochs=50,
    validation_data=(x_val, [y_val_split_onehot_value, y_val_split_onehot_color]),
    batch_size=128
)

In [ ]:
y_pred_value, y_pred_color = model_translearn_split.predict(x_test)
get_accuracy_split(y_test_split, y_pred_value, y_pred_color)

In [ ]:
plot_historyv2(history)
y_test_value = y_test_split[:,0]
y_test_color = y_test_split[:,1]
confusion_matrix(y_test_value, np.argmax(y_pred_value, axis=1), card_number)

In [ ]:
confusion_matrix(y_test_color, np.argmax(y_pred_color, axis=1), card_type)

### Transfer learning avec 2 outputs et fine-tuning

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(image_size, image_size, 3))

conv_base.trainable = False

input_layer = Input(shape=(image_size, image_size, 3))
x = conv_base(input_layer, training=False)
x = Flatten()(x)
prev_layer = Dense(512, activation="relu")(x)
prev_layer = Dense(256, activation="relu")(x)

output_value = Dense(8, activation="softmax", name="value")(prev_layer)
output_color = Dense(4, activation="softmax", name="color")(prev_layer)

output = [output_value, output_color]
model_translearn_split = Model(input_layer, output)

model_translearn_split.summary()

In [ ]:
loss_weights

In [ ]:
initial_epochs = 20

model_translearn_split.compile(optimizer=optimizers.Adam(lr=1e-4),
              loss=loss,
              metrics=metrics,
              loss_weights=loss_weights)

history = model_translearn_split.fit(x_train,
    [y_train_split_onehot_value, y_train_split_onehot_color],
    epochs=initial_epochs,
    batch_size=128,
    validation_data=(x_val, [y_val_split_onehot_value, y_val_split_onehot_color]))

In [ ]:
y_pred_value, y_pred_color = model_translearn_split.predict(x_test)
get_accuracy_split(y_test_split, y_pred_value, y_pred_color)

In [ ]:
conv_base.trainable = True

# Fonction pour dégeler progressivement les couches et réentraîner le modèle
def fine_tune_model_split(model, conv_base, x_train, y_train, x_val, y_val, initial_epochs, additional_epochs, learning_rate, layers_to_unfreeze):
    # Dégeler les couches
    conv_base.trainable = True
    for layer in conv_base.layers[:-layers_to_unfreeze]:
        layer.trainable = False

    # Recompiler le modèle avec un taux d'apprentissage plus bas
    model.compile(optimizer=optimizers.Adam(lr=1e-4),
                  loss=loss,
                  metrics=metrics,
                  loss_weights=loss_weights)

    # Continuer l'entraînement
    history_fine = model.fit(x_train, y_train,
                    validation_data=(x_val, y_val),
                    epochs=initial_epochs + additional_epochs,
                    steps_per_epoch=len(x_train) // 32,
                    validation_steps=len(x_val) // 32,
                    callbacks=[checkpoint, reduce_lr])

    return history_fine

# Définir les paramètres pour le fine-tuning progressif

total_epochs = 50
fine_tuning_steps = [
    {"epochs": 10, "learning_rate": 1e-5, "layers_to_unfreeze": 50},
    {"epochs": 10, "learning_rate": 1e-6, "layers_to_unfreeze": 100},
    {"epochs": 10, "learning_rate": 1e-7, "layers_to_unfreeze": len(conv_base.layers)}
]
# Exécuter le fine-tuning progressif
current_epochs = initial_epochs
for step in fine_tuning_steps:
    history_fine = fine_tune_model_split(model_translearn_split, conv_base, x_train, [y_train_split_onehot_value, y_train_split_onehot_color], x_val, [y_val_split_onehot_value, y_val_split_onehot_color],
                                   current_epochs, step["epochs"], step["learning_rate"], step["layers_to_unfreeze"])
    current_epochs += step["epochs"]


In [ ]:
y_pred_value, y_pred_color = model_translearn_split.predict(x_test)
get_accuracy_split(y_test_split, y_pred_value, y_pred_color)

In [ ]:
y_test_value = y_test_split[:,0]
y_test_color = y_test_split[:,1]
confusion_matrix(y_test_value, np.argmax(y_pred_value, axis=1), card_number)

In [ ]:
confusion_matrix(y_test_color, np.argmax(y_pred_color, axis=1), card_type)

### Transfer learning avec 2 outputs, augmentation et fine-tuning

In [ ]:
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(image_size, image_size, 3))

conv_base.trainable = False

input_layer = Input(shape=(image_size, image_size, 3))
x = conv_base(input_layer, training=False)
x = Flatten()(x)
prev_layer = Dense(1024, activation="relu")(x)
prev_layer = Dense(256, activation="relu")(x)

output_value = Dense(8, activation="softmax", name="value")(prev_layer)
output_color = Dense(4, activation="softmax", name="color")(prev_layer)

output = [output_value, output_color]
model_translearn_split_aug = Model(input_layer, output)

model_translearn_split_aug.summary()

In [ ]:
from tensorflow.keras.utils import Sequence

class CardSequence(Sequence):
    def __init__(self, x_set, y_value, y_color, batch_size, augmentations):
        self.x = x_set
        self.y_value = y_value
        self.y_color = y_color
        self.batch_size = batch_size
        self.augment = augmentations
        self.indices = np.arange(x_set.shape[0])
        np.random.shuffle(self.indices)

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def apply_augmentation(self, bx):
        batch_x = np.zeros((bx.shape[0], 128, 128, 3))
        for i in range(len(bx)):
            img = bx[i].astype('float32')
            transformed = self.augment(image=img.astype('float32'))
            batch_x[i] = transformed['image']
        return batch_x

    def __getitem__(self, idx):
        batch_indices = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_x = self.x[batch_indices]
        batch_y_value = self.y_value[batch_indices]
        batch_y_color = self.y_color[batch_indices]

        batch_x = self.apply_augmentation(batch_x)
        return np.array(batch_x), [batch_y_value, batch_y_color]

    def on_epoch_end(self):
        np.random.shuffle(self.indices)

train_gen_split = CardSequence(x_train, y_train_split_onehot_value, y_train_split_onehot_color, 32, transforms)

In [ ]:
initial_epochs = 20

model_translearn_split_aug.compile(optimizer=optimizers.Adam(lr=1e-4),
              loss=loss,
              metrics=metrics,
              loss_weights=loss_weights)

history = model_translearn_split_aug.fit(train_gen_split,
    epochs=initial_epochs,
    batch_size=128,
    validation_data=(x_val, [y_val_split_onehot_value, y_val_split_onehot_color]))

In [ ]:
y_pred_value, y_pred_color = model_translearn_split_aug.predict(x_test)
get_accuracy_split(y_test_split, y_pred_value, y_pred_color)

In [ ]:
conv_base.trainable = True

# Fonction pour dégeler progressivement les couches et réentraîner le modèle
def fine_tune_model_split(model, conv_base, x_train_generator, y_train, y_val, additional_epochs, learning_rate):
    # Dégeler les couches
    conv_base.trainable = True

    # Recompiler le modèle avec un taux d'apprentissage plus bas
    model.compile(optimizer=optimizers.Adam(lr=1e-4),
                  loss=loss,
                  metrics=metrics,
                  loss_weights=loss_weights)

    model.summary()

    # Continuer l'entraînement
    history_fine = model.fit(x_train_generator,
                    validation_data=(x_val, y_val),
                    epochs=initial_epochs + additional_epochs,
                    steps_per_epoch=len(x_train) // 32,
                    validation_steps=len(x_val) // 32,
                    callbacks=[checkpoint, reduce_lr])

    return history_fine

# Définir les paramètres pour le fine-tuning progressif

total_epochs = 50
fine_tuning_steps = [
    {"epochs": 25, "learning_rate": 1e-5, "layers_to_unfreeze": 50},
    {"epochs": 25, "learning_rate": 1e-6, "layers_to_unfreeze": 250},
    {"epochs": 25, "learning_rate": 1e-7, "layers_to_unfreeze": len(conv_base.layers)}
]
# Exécuter le fine-tuning progressif
for step in fine_tuning_steps:
    history_fine = fine_tune_model_split(model_translearn_split_aug, conv_base, train_gen_split, x_val, [y_val_split_onehot_value, y_val_split_onehot_color],
                                  step["epochs"], step["learning_rate"])
    current_epochs += step["epochs"]


In [ ]:
y_pred_value, y_pred_color = model_translearn_split_aug.predict(x_test)
get_accuracy_split(y_test_split, y_pred_value, y_pred_color)

In [ ]:
y_test_value = y_test_split[:,0]
y_test_color = y_test_split[:,1]
confusion_matrix(y_test_value, np.argmax(y_pred_value, axis=1), card_number)

In [ ]:
confusion_matrix(y_test_color, np.argmax(y_pred_color, axis=1), card_type)